In [1]:
import pandas as pd
import numpy as np
#from imblearn.over_sampling import RandomOverSampler
from xgboost import XGBClassifier
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

In [3]:
import seaborn as sns
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
train, test = pd.read_csv('train2.csv'), pd.read_csv('test2.csv')

In [5]:
train.head()

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,default_ind
0,230032,1696.0,1.6541,0.000,0.0,0.0,0.0,6015.0,322.0,40369.0,...,1.0,73.780,82.547,0.08696,10.0,0.63899,NaN,0.0,C,0
1,230033,1846.0,0.8095,0.000,0.0,0.0,102.0,7532.0,3171.0,18234.0,...,0.0,99.129,NaN,0.00000,13.0,0.63836,NaN,NaN,L,1
2,230034,1745.0,0.4001,0.000,0.0,0.0,NaN,2536.0,NaN,NaN,...,0.0,NaN,29.290,0.00000,1.0,1.00000,NaN,0.0,C,1
3,230035,1739.0,0.2193,0.000,0.0,0.0,1982.0,26440.0,4955.0,20316.0,...,0.0,96.272,NaN,0.15385,3.0,0.53241,0.0,0.0,L,0
4,230036,1787.0,0.0118,0.225,0.0,0.0,5451.0,5494.0,5494.0,7987.0,...,0.0,115.019,NaN,0.00000,1.0,0.92665,NaN,NaN,L,0


In [6]:
train = train.replace(['L', 'C'], [1.0, 0.0])

In [7]:
test.head()

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar38,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47
0,350053,1753.0,0.5001,0.000,0.0,0.0,206.0,19179.0,206.0,14221.0,...,1.0,0.0,NaN,20.551,0.42105,3.0,0.85661,NaN,0.0,L
1,350054,1894.0,1.9701,0.000,0.0,0.0,297.0,19820.0,19820.0,176895.0,...,3.0,0.0,NaN,63.047,0.00000,15.0,0.94391,0.0,0.0,C
2,350055,1774.0,0.1718,0.000,0.0,0.0,197.0,2563.0,877.0,8869.0,...,1.0,0.0,NaN,83.797,0.33333,3.0,0.76467,0.0,0.0,C
3,350056,1586.0,0.1123,5.299,0.0,0.0,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,1.50000,0.0,NaN,NaN,NaN,C
4,350057,1832.0,1.4442,0.000,0.0,0.0,179.0,7577.0,179.0,21059.0,...,3.0,0.0,96.052,66.665,0.00000,23.0,0.79190,0.0,0.0,L


In [8]:
test = test.replace(['L', 'C'], [1.0, 0.0])

In [9]:
train.head()

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,default_ind
0,230032,1696.0,1.6541,0.000,0.0,0.0,0.0,6015.0,322.0,40369.0,...,1.0,73.780,82.547,0.08696,10.0,0.63899,NaN,0.0,0.0,0
1,230033,1846.0,0.8095,0.000,0.0,0.0,102.0,7532.0,3171.0,18234.0,...,0.0,99.129,NaN,0.00000,13.0,0.63836,NaN,NaN,1.0,1
2,230034,1745.0,0.4001,0.000,0.0,0.0,NaN,2536.0,NaN,NaN,...,0.0,NaN,29.290,0.00000,1.0,1.00000,NaN,0.0,0.0,1
3,230035,1739.0,0.2193,0.000,0.0,0.0,1982.0,26440.0,4955.0,20316.0,...,0.0,96.272,NaN,0.15385,3.0,0.53241,0.0,0.0,1.0,0
4,230036,1787.0,0.0118,0.225,0.0,0.0,5451.0,5494.0,5494.0,7987.0,...,0.0,115.019,NaN,0.00000,1.0,0.92665,NaN,NaN,1.0,0


In [10]:
train_default = train[train['default_ind'] == 1]
train_normal = train[train['default_ind'] == 0]

In [12]:
len(train_normal)

60303

In [13]:
train_normal_und = train_normal.sample(n=30000, random_state=2)

In [15]:
train_normal_und.head()

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,default_ind
67970,298002,NaN,0.0000,2.891,0.0,0.000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.00000,0.0,NaN,NaN,NaN,0.0,0
27302,257334,1834.0,0.1301,0.000,0.0,0.000,1163.0,15471.0,15471.0,105981.0,...,0.0,NaN,NaN,0.00000,14.0,0.72687,0.0,0.0,1.0,0
30665,260697,1730.0,0.7997,14.354,0.0,2.817,59.0,3967.0,615.0,39109.0,...,0.0,91.399,NaN,0.21951,17.0,0.15199,NaN,NaN,0.0,0
50567,280599,1700.0,3.2464,1.536,0.0,0.000,NaN,431.0,NaN,743.0,...,0.0,NaN,NaN,0.33333,2.0,1.00000,NaN,NaN,1.0,0
45078,275110,1782.0,0.3619,0.000,0.0,0.000,428.0,9458.0,1057.0,26162.0,...,0.0,NaN,NaN,0.07143,6.0,0.47924,NaN,0.0,0.0,0


In [21]:
train_und = pd.concat([train_normal_und, train_default]).sample(frac=1, random_state=3).reset_index(drop=True)
train_und

,application_key,mvar1,mvar2,mvar3,mvar4,mvar5,mvar6,mvar7,mvar8,mvar9,...,mvar39,mvar40,mvar41,mvar42,mvar43,mvar44,mvar45,mvar46,mvar47,default_ind
0,245608,1616.0,1.2171,9.235,0.000,0.000,NaN,0.0,NaN,NaN,...,0.0,100.000,NaN,1.00000,1.0,0.55386,NaN,NaN,0.0,1
1,238844,1876.0,0.9501,0.000,0.000,0.000,426.0,6442.0,6442.0,19820.0,...,0.0,NaN,NaN,0.00000,2.0,1.00000,NaN,NaN,1.0,0
2,271453,1770.0,6.9585,0.637,0.000,0.000,539.0,9081.0,1560.0,7036.0,...,0.0,93.616,59.469,0.35294,12.0,0.49653,NaN,0.0,0.0,0
3,255711,1763.0,3.1145,0.000,0.000,0.000,163.0,5984.0,4955.0,57141.0,...,0.0,NaN,97.610,0.00000,29.0,0.47112,NaN,0.0,0.0,0
4,243775,NaN,0.8701,0.000,0.000,0.000,NaN,NaN,NaN,367.0,...,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,0.0,1
5,233452,1586.0,0.0868,22.965,0.000,0.000,NaN,NaN,NaN,622.0,...,NaN,NaN,NaN,0.75000,0.0,NaN,NaN,0.0,0.0,0
6,237417,NaN,0.3723,5.948,0.000,0.000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.14286,0.0,NaN,NaN,NaN,1.0,1
7,309698,1912.0,0.0000,0.000,0.000,0.000,3964.0,39179.0,12111.0,61442.0,...,0.0,NaN,NaN,0.00000,10.0,0.90014,0.0,0.0,1.0,0
8,296552,1904.0,NaN,0.000,0.000,0.000,1441.0,14865.0,14865.0,260113.0,...,0.0,NaN,NaN,0.00000,17.0,0.30742,NaN,NaN,0.0,0
9,266447,1723.0,0.4018,1.296,0.000,0.000,NaN,6637.0,NaN,71104.0,...,0.0,NaN,NaN,0.12000,4.0,1.00000,0.0,0.0,0.0,0


In [25]:
train_und['default_ind'].value_counts()*100/len(train_und)

0    60.365817
1    39.634183
Name: default_ind, dtype: float64

In [26]:
train_X = train_und.iloc[:,1:-1]
train_Y = train_und.iloc[:,-1]
test_X = test.iloc[:,1:]

In [23]:
sum(train_Y)

19697

In [29]:
xgb_model = XGBClassifier(n_jobs = 6)

parameters = {'max_depth': [3,5,7,9,12],
              'n_estimators': [50, 70, 100, 150, 200, 300], 
              'seed': [42]}

clf = GridSearchCV(xgb_model, param_grid=parameters, scoring='recall', verbose=5)


clf.fit(train_X, train_Y)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
[CV] max_depth=3, n_estimators=50, seed=42 ...........................
[CV]  max_depth=3, n_estimators=50, seed=42, score=0.6704233932378921, total=   1.9s
[CV] max_depth=3, n_estimators=50, seed=42 ...........................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.0s remaining:    0.0s


[CV]  max_depth=3, n_estimators=50, seed=42, score=0.6643314042034725, total=   1.7s
[CV] max_depth=3, n_estimators=50, seed=42 ...........................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.9s remaining:    0.0s


[CV]  max_depth=3, n_estimators=50, seed=42, score=0.6642802741812642, total=   1.7s
[CV] max_depth=3, n_estimators=70, seed=42 ...........................


...

[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    8.3s remaining:    0.0s


[CV]  max_depth=3, n_estimators=70, seed=42, score=0.6670727992689613, total=   2.4s
[CV] max_depth=3, n_estimators=70, seed=42 ...........................
[CV]  max_depth=3, n_estimators=70, seed=42, score=0.6668697638994668, total=   2.5s
[CV] max_depth=3, n_estimators=100, seed=42 ..........................
[CV]  max_depth=3, n_estimators=100, seed=42, score=0.6708802924154736, total=   3.6s
[CV] max_depth=3, n_estimators=100, seed=42 ..........................
[CV]  max_depth=3, n_estimators=100, seed=42, score=0.6690526957051477, total=   3.3s
[CV] max_depth=3, n_estimators=100, seed=42 ..........................
[CV]  max_depth=3, n_estimators=100, seed=42, score=0.6691546077684691, total=   3.3s
[CV] max_depth=3, n_estimators=150, seed=42 ..........................
[CV]  max_depth=3, n_estimators=150, seed=42, score=0.6695095948827292, total=   5.0s
[CV] max_depth=3, n_estimators=150, seed=42 ..........................
[CV]  max_depth=3, n_estimators=150, seed=42, score=0.669966

[CV]  max_depth=9, n_estimators=70, seed=42, score=0.6631130063965884, total=   7.7s
[CV] max_depth=9, n_estimators=70, seed=42 ...........................
[CV]  max_depth=9, n_estimators=70, seed=42, score=0.6672250989948219, total=   7.6s
[CV] max_depth=9, n_estimators=70, seed=42 ...........................
[CV]  max_depth=9, n_estimators=70, seed=42, score=0.6676313785224677, total=   7.8s
[CV] max_depth=9, n_estimators=100, seed=42 ..........................
[CV]  max_depth=9, n_estimators=100, seed=42, score=0.6676819981724033, total=  10.9s
[CV] max_depth=9, n_estimators=100, seed=42 ..........................
[CV]  max_depth=9, n_estimators=100, seed=42, score=0.6707279926896131, total=  10.7s
[CV] max_depth=9, n_estimators=100, seed=42 ..........................
[CV]  max_depth=9, n_estimators=100, seed=42, score=0.6673267326732674, total=  10.8s
[CV] max_depth=9, n_estimators=150, seed=42 ..........................
[CV]  max_depth=9, n_estimators=150, seed=42, score=0.6658544

[Parallel(n_jobs=1)]: Done  90 out of  90 | elapsed: 19.0min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=6, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 5, 7, 9, 12], 'n_estimators': [50, 70, 100, 150, 200, 300], 'seed': [42]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='recall', verbose=5)

In [30]:
clf.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=6, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
       subsample=1)

In [32]:
predictions = clf.predict(test_X)
model = clf.best_estimator_

### Submission

In [33]:
def probability_dataframe(model,test_data, predictions):
    probabilities = model.predict_proba(test_data)
    d = {'application_key':test['application_key'], 'default_ind':predictions.tolist(),\
         'probability':probabilities.tolist()}

    prediction_df = pd.DataFrame(data = d)
    prediction_df['Confidence'] = prediction_df.apply(lambda row: row[2][row[1]], axis =1)
    prediction_df.sort_values(by = 'Confidence', axis = 0, ascending=False, inplace = True)
    prediction_df.reset_index(drop=True, inplace = True)
    
    return prediction_df

In [34]:
probability_df = probability_dataframe(model, test_X, predictions)
probability_df

,application_key,default_ind,probability,Confidence
0,368204,0,"[0.9928693175315857, 0.007130669429898262]",0.992869
1,355674,0,"[0.9928351044654846, 0.007164891343563795]",0.992835
2,367882,0,"[0.9928014874458313, 0.00719850929453969]",0.992801
3,365321,0,"[0.9927955269813538, 0.007204476278275251]",0.992796
4,350246,0,"[0.9927190542221069, 0.007280949503183365]",0.992719
5,365430,0,"[0.9926544427871704, 0.007345566060394049]",0.992654
6,367601,0,"[0.9925822019577026, 0.007417783606797457]",0.992582
7,370383,0,"[0.9925565719604492, 0.00744342477992177]",0.992557
8,367047,0,"[0.9925101399421692, 0.007489841431379318]",0.992510
9,352522,0,"[0.9924963116645813, 0.007503710221499205]",0.992496


In [35]:
probability_df.iloc[:10000]

,application_key,default_ind,probability,Confidence
0,368204,0,"[0.9928693175315857, 0.007130669429898262]",0.992869
1,355674,0,"[0.9928351044654846, 0.007164891343563795]",0.992835
2,367882,0,"[0.9928014874458313, 0.00719850929453969]",0.992801
3,365321,0,"[0.9927955269813538, 0.007204476278275251]",0.992796
4,350246,0,"[0.9927190542221069, 0.007280949503183365]",0.992719
5,365430,0,"[0.9926544427871704, 0.007345566060394049]",0.992654
6,367601,0,"[0.9925822019577026, 0.007417783606797457]",0.992582
7,370383,0,"[0.9925565719604492, 0.00744342477992177]",0.992557
8,367047,0,"[0.9925101399421692, 0.007489841431379318]",0.992510
9,352522,0,"[0.9924963116645813, 0.007503710221499205]",0.992496


In [ ]:
feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = train_X.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
feature_importances

In [36]:
sum(probability_df[:10000]['default_ind'])

1360

In [37]:
submission = probability_df.iloc[:,:2]
submission.to_csv('undersmpled_opt.csv', index = False, header=False)